In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import torch.nn.functional as F
from focal import FocalLoss
from volo import volo_d1
from sklearn.metrics import roc_auc_score
import torchvision.transforms as T
from src.res import wide_resnet101_2
from src.dense import densenet121
from src.inception import inception_v3
print(torch.__version__)
torch.manual_seed(0)

1.8.1+cu111


In [16]:
class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = densenet121(pretrained=False, **{'drop_rate' : 0.3})
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

model = DenseNet121(4)
model.load_state_dict(torch.load('./model_weights/dense_best.pt'))

<All keys matched successfully>

In [46]:
training = os.listdir('../train/')
study_label = pd.read_csv('../archive/train_study_level.csv')
image_label = pd.read_csv('../archive/train_image_level.csv')
paths = []
labels = []
dct = pd.read_csv('train.csv', index_col=0)
dct['image'] = dct.image.apply(lambda x: x[:-4])
dct = dct.set_index('study').to_dict()['image']

In [47]:
for index, row in study_label.iterrows():
    name = dct[row['id'].replace('_study', '')] + '.png'
    if name in training:
        paths.append('../train/' + name)
        if row['Negative for Pneumonia'] == 1:
            labels.append(0)
        elif row['Typical Appearance'] == 1:
            labels.append(1)
        elif row['Indeterminate Appearance'] == 1:
            labels.append(2)
        elif row['Atypical Appearance'] == 1:
            labels.append(3)
    else:
        print(name)

In [48]:
'''
class Net(nn.Module):
    def __init__(self, out_size, model):
        super(Net, self).__init__()
        if model == 'dense':
            self.model = torchvision.models.densenet121(pretrained=True, **{'drop_rate' : 0.3})
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(
                nn.Linear(num_ftrs, out_size)
            )
            
        elif model == 'res':
            self.model = torchvision.models.wide_resnet101_2(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Sequential(
                nn.Linear(num_ftrs, out_size)
            )
        elif model == 'inception':
            self.model = torchvision.models.inception_v3(pretrained=True, **{"aux_logits": False})
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Sequential(
                nn.Linear(num_ftrs, out_size)
            )
    def forward(self, x):
        x = self.model(x)
        return x
'''

'\nclass Net(nn.Module):\n    def __init__(self, out_size, model):\n        super(Net, self).__init__()\n        if model == \'dense\':\n            self.model = torchvision.models.densenet121(pretrained=True, **{\'drop_rate\' : 0.3})\n            num_ftrs = self.model.classifier.in_features\n            self.model.classifier = nn.Sequential(\n                nn.Linear(num_ftrs, out_size)\n            )\n            \n        elif model == \'res\':\n            self.model = torchvision.models.wide_resnet101_2(pretrained=True)\n            num_ftrs = self.model.fc.in_features\n            self.model.fc = nn.Sequential(\n                nn.Linear(num_ftrs, out_size)\n            )\n        elif model == \'inception\':\n            self.model = torchvision.models.inception_v3(pretrained=True, **{"aux_logits": False})\n            num_ftrs = self.model.fc.in_features\n            self.model.fc = nn.Sequential(\n                nn.Linear(num_ftrs, out_size)\n            )\n    def forward(s

In [49]:
class classification(nn.Module):
    def __init__(self, paths, labels, size=(512,512)):
        self.paths = paths
        self.labels = labels
        self.example = []
        self.size =size
    
    def __getitem__(self, idx):
            path = self.paths[idx]
            img = cv2.imread(path)[:, :, 0]
            img = (img - np.mean(img))/np.std(img)
            img = cv2.resize(img, self.size)
            x = torch.from_numpy(np.array(img)).view((1, self.size[0], self.size[1]))
            x = x.float()
            y = self.labels[idx]
            y = torch.tensor(y)
            return x, y
        
    def __len__(self):
        return len(self.paths)
    
    def get(self):
        return self.example

In [52]:
def roc_auc_compute_fn(y_preds: torch.Tensor, y_targets: torch.Tensor) -> float:

    y_true = y_targets.cpu().numpy()
    y_pred = y_preds.cpu().numpy()
    print(y_true.shape, y_pred.shape)
    return roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovo')

def one_hot(labels: torch.Tensor, num_classes: int, dtype: torch.dtype = torch.float, dim: int = 1) -> torch.Tensor:
    # if `dim` is bigger, add singleton dim at the end
    if labels.ndim < dim + 1:
        shape = list(labels.shape) + [1] * (dim + 1 - len(labels.shape))
        labels = torch.reshape(labels, shape)

    sh = list(labels.shape)

    if sh[dim] != 1:
        raise AssertionError("labels should have a channel with length equal to one.")

    sh[dim] = num_classes

    o = torch.zeros(size=sh, dtype=dtype, device=labels.device)
    labels = o.scatter_(dim=dim, index=labels.long(), value=1)

    return labels

In [53]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [54]:
dataset = classification(paths, labels, size=(512, 512))
print(len(dataset))
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                            batch_size=16,
                                            shuffle=False)

6054


In [55]:
model.to(device)
model.eval()    

test_output = []
test_true = []
    
for i, (data,labels) in tqdm(enumerate(train_loader),
                                       total = len(train_loader)):
    data, labels = data.to(device),labels.to(device)

    with torch.no_grad():
        outputs = model(data)

    outputs = torch.nn.functional.softmax(outputs, 1)
    _, predicted = torch.max(outputs.data, 1)
    test_output.append(predicted.detach().cpu().numpy())
    test_true.append(labels.detach().cpu().numpy())
    

test_output = np.concatenate(test_output)
test_true = np.concatenate(test_true)

In [56]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_output, test_true, normalize='true')

array([[0.67883996, 0.10741139, 0.14070892, 0.07303974],
       [0.08797269, 0.66911765, 0.16780462, 0.07510504],
       [0.20052083, 0.27864583, 0.38541667, 0.13541667],
       [0.        , 0.        , 0.        , 0.        ]])